In [3]:
# Draft Analysis: YouTube-Kanal Performance Analyse für ntv

# Import der benötigten Bibliotheken
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, classification_report

In [4]:
# Data Loading
def load_data():
    """
    Lädt die YouTube Analytics Daten
    """
    # URLs aktualisieren falls nötig
    analytics_path = 'https://raw.githubusercontent.com/Takual/project/main/data/raw/youtube%20analytics%20ntv.csv'
    
    try:
        # Daten laden mit korrektem Encoding und Delimiter
        df = pd.read_csv(analytics_path, delimiter=';', encoding='ISO-8859-1')
        
        # Spalten bereinigen
        df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
        
        return df
    
    except Exception as e:
        print(f"Fehler beim Laden der Daten: {e}")
        return None


In [5]:

# Data Preparation
def prepare_data(df):
    """
    Vorbereitung der Daten für die Analyse
    """
    # Fehlende Werte behandeln
    df.dropna(subset=['aufrufe', 'wiedergabezeit_(stunden)'], inplace=True)
    
    # Erfolgs-Kategorisierung
    def categorize_success(row):
        aufrufe = row['aufrufe']
        wiedergabezeit = row['wiedergabezeit_(stunden)']
        
        if aufrufe > df['aufrufe'].quantile(0.75) and wiedergabezeit > df['wiedergabezeit_(stunden)'].quantile(0.75):
            return 'sehr erfolgreich'
        elif aufrufe > df['aufrufe'].quantile(0.5) and wiedergabezeit > df['wiedergabezeit_(stunden)'].quantile(0.5):
            return 'erfolgreich'
        else:
            return 'weniger erfolgreich'
    
    df['erfolgskategorie'] = df.apply(categorize_success, axis=1)
    
    # Kategorische Variablen enkodieren
    le = LabelEncoder()
    df['thema_encoded'] = le.fit_transform(df['thema'])
    df['gestaltung_thumbnail_encoded'] = le.fit_transform(df['gestaltung_thumbnail'])
    
    return df


In [6]:

# Explorative Datenanalyse
def perform_eda(df):
    """
    Explorative Datenanalyse mit Visualisierungen
    """
    plt.figure(figsize=(15, 10))
    
    # Histogramme für Aufrufe und Wiedergabezeit
    plt.subplot(2, 2, 1)
    df['aufrufe'].hist(bins=30)
    plt.title('Verteilung der Aufrufe')
    
    plt.subplot(2, 2, 2)
    df['wiedergabezeit_(stunden)'].hist(bins=30)
    plt.title('Verteilung der Wiedergabezeit')
    
    # Boxplot für Erfolgskategorien
    plt.subplot(2, 2, 3)
    sns.boxplot(x='erfolgskategorie', y='aufrufe', data=df)
    plt.title('Aufrufe nach Erfolgskategorie')
    
    # Streudiagramm Aufrufe vs. Wiedergabezeit
    plt.subplot(2, 2, 4)
    plt.scatter(df['aufrufe'], df['wiedergabezeit_(stunden)'])
    plt.xlabel('Aufrufe')
    plt.ylabel('Wiedergabezeit (Stunden)')
    plt.title('Aufrufe vs. Wiedergabezeit')
    
    plt.tight_layout()
    plt.savefig('eda_visualisierungen.png')
    plt.close()



In [7]:
# Modellierung
def build_models(df):
    """
    Modelle für Vorhersage von Aufrufen und Erfolg
    """
    # Features vorbereiten
    features = ['länge_des_videos', 'thema_encoded', 
                'gestaltung_thumbnail_encoded', 
                'bewertung_titel']
    
    X = df[features]
    y_aufrufe = df['aufrufe']
    y_erfolg = df['erfolgskategorie']
    
    # Train-Test Split
    X_train, X_test, y_aufrufe_train, y_aufrufe_test = train_test_split(X, y_aufrufe, test_size=0.2)
    _, _, y_erfolg_train, y_erfolg_test = train_test_split(X, y_erfolg, test_size=0.2)
    
    # Skalierung
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Lineare Regression für Aufrufe
    lr_model = LinearRegression()
    lr_model.fit(X_train_scaled, y_aufrufe_train)
    y_pred_aufrufe = lr_model.predict(X_test_scaled)
    
    print("Lineare Regression - Aufrufe:")
    print(f"R² Score: {r2_score(y_aufrufe_test, y_pred_aufrufe)}")
    print(f"Mean Squared Error: {mean_squared_error(y_aufrufe_test, y_pred_aufrufe)}")
    
    # Logistische Regression für Erfolg
    lr_erfolg = LogisticRegression(multi_class='ovr')
    lr_erfolg.fit(X_train_scaled, y_erfolg_train)
    y_pred_erfolg = lr_erfolg.predict(X_test_scaled)
    
    print("\nLogistische Regression - Erfolgskategorien:")
    print(classification_report(y_erfolg_test, y_pred_erfolg))
    
    # Modelle speichern
    import joblib
    joblib.dump(lr_model, 'models/aufrufe_model.pkl')
    joblib.dump(lr_erfolg, 'models/erfolg_model.pkl')


In [ ]:

def main():
    # Hauptausführung
    df = load_data()
    if df is not None:
        df_prepared = prepare_data(df)
        perform_eda(df_prepared)
        build_models(df_prepared)

if __name__ == "__main__":
    main()